In [ ]:
#!jupyter nbconvert --to=python Biomass_Maredat_comparison.ipynb

In [7]:
class Bio_map_all:
    '''
    class Bio_map_all(runname,resultpath,savepath,meshpath,first_year,last_year,
                 mapproj='pc',cmap,cmap_extension,savefig=False, verbose=False, output=False, 
                            plotting=True, Taylor=True)
    '''
    
    def __init__(self,resultpath,savepath,mesh,meshpath,first_year,last_year,
                 mapproj='pc',cmap='RdYlGn',cmap_extension='max',savefig=False,output=False,plotting=True,verbose=False,Taylor=True,runname='fesom'):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.mesh = mesh
        self.meshpath = meshpath
        self.fyear = first_year
        self.lyear = last_year
        self.mapproj = mapproj
        self.cmap = cmap
        self.cmap_extension = cmap_extension
        self.savefig = savefig
        self.verbose = verbose
        self.output = output
        self.plotting = plotting
        self.Taylor = Taylor
        

        import matplotlib.pyplot as plt
        import matplotlib.colors as colors
        import numpy as np
        from scipy.interpolate import griddata
        import skill_metrics as sm
        import cartopy.crs as ccrs
        import scipy.io as spio
        import cartopy.feature as cfeature
        from netCDF4 import Dataset
        import pyfesom2 as pf
        from pathlib import Path
        
        if self.mapproj == 'rob':
            box=[-180, 180, -90, 90]
        elif self.mapproj == 'pc':
            box=[-180, 180, -90, 90]
        elif self.mapproj == 'sp':
            box=[-180, 180, -90, -30]
        elif self.mapproj == 'np':
            box=[-180, 180, 60, 90]
            
        self.mapproj = pf.get_proj(self.mapproj)

        if(self.verbose):
            print('Processing {0}'.format(self.resultpath))
            
            
        years = np.arange(self.fyear, self.lyear+1,1)
        
        # Load biomass data and compute layersum and volsum -------------------------------------------------------------------------------
            
        # Diatoms  
        DiaC = pf.get_data(resultpath, 'DiaC', years, mesh, runid=self.runname, how="mean", compute=True)
        DiaC = DiaC * 12.01
        DiaC_layersum = pf.layersum_data(DiaC,mesh,uplow=[0, 6250])
        DiaC_layersum = DiaC_layersum/1e3 # mg/m2 -> g/m2
        DiaC_volsum   = pf.volsum_data(DiaC,mesh,uplow=[0, 6250])
        DiaC_volsum   = DiaC_volsum/1e18 # mg -> Pg
        
        # Small phytoplankton
        SPC = pf.get_data(resultpath, 'PhyC', years, mesh, runid=self.runname, how="mean", compute=True)
        SPC = SPC * 12.01
        SPC_layersum = pf.layersum_data(SPC,mesh,uplow=[0, 6250])
        SPC_layersum = SPC_layersum/1e3 # mg/m2 -> g/m2
        SPC_volsum   = pf.volsum_data(SPC,mesh,uplow=[0, 6250])
        SPC_volsum   = SPC_volsum/1e18 # mg -> Pg
        
        # Coccolithophores
        cocco_path = Path(self.resultpath + '/CoccoC.fesom.'+str(years[0])+'.nc') 
        if cocco_path.is_file():
            CoccoC = pf.get_data(resultpath, 'CoccoC', years, mesh, runid=self.runname, how="mean", compute=True)
            CoccoC = CoccoC * 12.01
            CoccoC_layersum = pf.layersum_data(CoccoC,mesh,uplow=[0, 6250])
            CoccoC_layersum = CoccoC_layersum/1e3 # mg/m2 -> g/m2
            CoccoC_volsum   = pf.volsum_data(CoccoC,mesh,uplow=[0, 6250])
            CoccoC_volsum   = CoccoC_volsum/1e18 # mg -> Pg

        # Phaeocystis 
        phaeo_path = Path(self.resultpath + '/PhaeoC.fesom.'+str(years[0])+'.nc') 
        if phaeo_path.is_file():
            PhaeoC = pf.get_data(resultpath, 'PhaeoC', years, mesh, runid=self.runname, how="mean", compute=True)
            PhaeoC = PhaeoC * 12.01
            PhaeoC_layersum = pf.layersum_data(PhaeoC,mesh,uplow=[0, 6250])
            PhaeoC_layersum = PhaeoC_layersum/1e3 # mg/m2 -> g/m2
            PhaeoC_volsum   = pf.volsum_data(PhaeoC,mesh,uplow=[0, 6250])
            PhaeoC_volsum   = PhaeoC_volsum/1e18 # mg -> Pg

        
        # Microzooplankton
        micro_path = Path(self.resultpath + '/Zoo3C.fesom.'+str(years[0])+'.nc') 
        if micro_path.is_file():
            MicroC = pf.get_data(resultpath, 'Zoo3C', years, mesh, runid=self.runname, how="mean", compute=True)
            MicroC = MicroC * 12.01
            MicroC_layersum = pf.layersum_data(MicroC,mesh,uplow=[0, 6250])
            MicroC_layersum = MicroC_layersum/1e3 # mg/m2 -> g/m2
            MicroC_volsum   = pf.volsum_data(MicroC,mesh,uplow=[0, 6250])
            MicroC_volsum   = MicroC_volsum/1e18 # mg -> Pg
        
        # Mesozooplankton
        MesoC = pf.get_data(resultpath, 'HetC', years, mesh, runid=self.runname, how="mean", compute=True)
        MesoC = MesoC * 12.01
        MesoC_layersum = pf.layersum_data(MesoC,mesh,uplow=[0, 6250])
        MesoC_layersum = MesoC_layersum/1e3 # mg/m2 -> g/m2
        MesoC_volsum   = pf.volsum_data(MesoC,mesh,uplow=[0, 6250])
        MesoC_volsum   = MesoC_volsum/1e18 # mg -> Pg

        # Macrozooplankton
        macro_path = Path(self.resultpath + '/Zoo2C.fesom.'+str(years[0])+'.nc') 
        if macro_path.is_file():
            MacroC = pf.get_data(resultpath, 'Zoo2C', years, mesh, runid=self.runname, how="mean", compute=True)
            MacroC = MacroC * 12.01
            MacroC_layersum = pf.layersum_data(MacroC,mesh,uplow=[0, 6250])
            MacroC_layersum = MacroC_layersum/1e3 # mg/m2 -> g/m2
            MacroC_volsum   = pf.volsum_data(MacroC,mesh,uplow=[0, 6250])
            MacroC_volsum   = MacroC_volsum/1e18 # mg -> Pg
            
            
            
        # Plot ---------------------------------------------------------------------------------------------------------
        
        levels_diatoms            = np.arange(0,6,0.3)
        levels_smallphytoplankton = np.arange(0,6,0.3)
        levels_coccolithophores   = np.arange(0,4,0.4)
        levels_phaeocystis        = np.arange(0,6,0.3)
        levels_microzooplankton   = np.arange(0,1,0.1)
        levels_mesozooplankton    = np.arange(0,0.5,0.05)
        levels_macrozooplankton   = np.arange(0,0.5,0.05)
        unit_phytoplankton        = 'Biomass [g m$^{-2}$]'
        unit_zooplankton          = 'Biomass [g m$^{-2}$]'
        cmap_extension            = 'max'
        cmap                      = 'viridis'
        mapproj                   = 'pc'
        mapproj                   = pf.get_proj(mapproj)
        box                       = [-180, 180, -90, 90]
        

        fig, axes = plt.subplots(4,2,
                             gridspec_kw={'hspace': 0.2, 'wspace': 0.1},
                             subplot_kw=dict(projection=mapproj),
                             figsize=(10,14))

        # Diatoms
        m1 = axes[0,0]
        f1 = pf.subplot(mesh, fig, m1, [DiaC_layersum[:]], levels = levels_diatoms,
                        units = unit_phytoplankton, mapproj = mapproj, cmap = cmap, cmap_extension = cmap_extension, box = box)
        m1.set_title('Diatoms', fontsize=12)


        cbar1_ax = fig.add_axes([0.135, 0.715, 0.35, 0.01])  # left, bottom, width, height
        cbar1 = fig.colorbar(f1,
                        cax = cbar1_ax, 
                        orientation = 'horizontal',
                        extend = 'max',
                        #ticks = [0,4,8,12,16,20,24,28,32,36,40]
                            ) 
        cbar1.ax.tick_params(labelsize=10)
        cbar1.set_label(unit_phytoplankton, fontsize=10)
        
        DiaC_volsum_text = str(round(DiaC_volsum[0],2))
        DiaC_text_all = r'$\Sigma$ '+DiaC_volsum_text+' Pg C'
        m1.text(0.7,0.03,DiaC_text_all, transform=m1.transAxes, color='white', fontsize=10, fontweight='bold')
        
        
        # Small phytoplankton
        m2 = axes[0,1]
        f2 = pf.subplot(mesh, fig, m2, [SPC_layersum[:]], levels = levels_smallphytoplankton,
                        units = unit_phytoplankton, mapproj = mapproj, cmap = cmap, cmap_extension = cmap_extension, box = box)
        m2.set_title('Small phytoplankton', fontsize=12)

        cbar2_ax = fig.add_axes([0.535, 0.715, 0.35, 0.01])  # left, bottom, width, height
        cbar2 = fig.colorbar(f2,
                        cax = cbar2_ax, 
                        orientation = 'horizontal',
                        extend = 'max',
                        #ticks = [0,4,8,12,16,20,24,28,32,36]
                            ) 
        cbar2.ax.tick_params(labelsize=10)
        cbar2.set_label(unit_phytoplankton, fontsize=10)
        
        SPC_volsum_text = str(round(SPC_volsum[0],2))
        SPC_text_all = r'$\Sigma$ '+SPC_volsum_text+' Pg C'
        m2.text(0.7,0.03,SPC_text_all, transform=m2.transAxes, color='white', fontsize=10, fontweight='bold')
        
        
        # Coccolithophores
        m3 = axes[1,0]
        if cocco_path.is_file():
            f3 = pf.subplot(mesh, fig, m3, [CoccoC_layersum[:]], levels = levels_coccolithophores,
                            units = unit_phytoplankton, mapproj = mapproj, cmap = cmap, cmap_extension = cmap_extension, box = box)
            m3.set_title('Coccolithophores', fontsize=12)


            cbar3_ax = fig.add_axes([0.135, 0.515, 0.35, 0.01])  # left, bottom, width, height  
            cbar3 = fig.colorbar(f3,
                            cax = cbar3_ax, 
                            orientation = 'horizontal',
                            extend = 'max',
                            #ticks = [0,4,8,12,16,20,24,28,32,36]
                                ) 
            cbar3.ax.tick_params(labelsize=10)
            cbar3.set_label(unit_phytoplankton, fontsize=10)
            
            CoccoC_volsum_text = str(round(CoccoC_volsum[0],2))
            CoccoC_text_all = r'$\Sigma$ '+CoccoC_volsum_text+' Pg C'
            m3.text(0.7,0.03,CoccoC_text_all, transform=m3.transAxes, color='white', fontsize=10, fontweight='bold')
        
        
        elif not cocco_path.is_file() and not phaeo_path.is_file():
            axes[0,2].remove()

        # Phaeocystis
        if phaeo_path.is_file():
            m7 = axes[1,1]
            f7 = pf.subplot(mesh, fig, m7, [PhaeoC_layersum[:]], levels = levels_phaeocystis,
                            units = unit_phytoplankton, mapproj = mapproj, cmap = cmap, cmap_extension = cmap_extension, box = box)
            m7.set_title('Phaeocystis', fontsize=12)
            
            cbar7_ax = fig.add_axes([0.535, 0.515, 0.35, 0.01])  # left, bottom, width, height
            cbar7 = fig.colorbar(f7,
                            cax = cbar7_ax, 
                            orientation = 'horizontal',
                            extend = 'max',
                            #ticks = [0,4,8,12,16,20,24,28,32,36]
                                ) 
            cbar7.ax.tick_params(labelsize=10)
            cbar7.set_label(unit_phytoplankton, fontsize=10)
            
            PhaeoC_volsum_text = str(round(PhaeoC_volsum[0],2))
            PhaeoC_text_all = r'$\Sigma$ '+PhaeoC_volsum_text+' Pg C'
            m7.text(0.7,0.03,PhaeoC_text_all, transform=m7.transAxes, color='white', fontsize=10, fontweight='bold')

        else: 
            axes[1,1].remove            
                
        # Microzooplankton
        if phaeo_path.is_file():
            axis = axes[2,0]
        else:
            axis = axes[1,1]
        m4 = axis
        if micro_path.is_file():
            f4 = pf.subplot(mesh, fig, m4, [MicroC_layersum[:]], levels = levels_microzooplankton,
                            units = unit_phytoplankton, mapproj = mapproj, cmap = cmap, cmap_extension = cmap_extension, box = box)
            m4.set_title('Microzooplankton', fontsize=12)
            
            if phaeo_path.is_file():
                cbar4_ax = fig.add_axes([0.135, 0.315, 0.35, 0.01])  # left, bottom, width, height
            else:
                cbar4_ax = fig.add_axes([0.535, 0.515, 0.35, 0.01])  # left, bottom, width, height
            cbar4 = fig.colorbar(f4,
                            cax = cbar4_ax, 
                            orientation = 'horizontal',
                            extend = 'max',
                            #ticks = [0,4,8,12,16,20,24,28,32,36]
                                ) 
            cbar4.ax.tick_params(labelsize=10)
            cbar4.set_label(unit_zooplankton, fontsize=10)
            
            MicroC_volsum_text = str(round(MicroC_volsum[0],2))
            MicroC_text_all = r'$\Sigma$ '+MicroC_volsum_text+' Pg C'
            m4.text(0.7,0.03,MicroC_text_all, transform=m4.transAxes, color='white', fontsize=10, fontweight='bold')
            
        else:
            axis.remove()
        
        
        # Mesozooplankton
        if phaeo_path.is_file() & micro_path.is_file():
            axis = axes[2,1]
        else:
            axis = axes[2,0]
            
        m5 = axis
        f5 = pf.subplot(mesh, fig, m5, [MesoC_layersum[:]], levels = levels_mesozooplankton,
                        units = unit_phytoplankton, mapproj = mapproj, cmap = cmap, cmap_extension = cmap_extension, box = box)
        m5.set_title('Mesozooplankton', fontsize=12)

        if phaeo_path.is_file() & micro_path.is_file():
            cbar5_ax = fig.add_axes([0.535, 0.315, 0.35, 0.01])  # left, bottom, width, height
        else:
            cbar5_ax = fig.add_axes([0.135, 0.315, 0.35, 0.01])  # left, bottom, width, height
        cbar5 = fig.colorbar(f5,
                        cax = cbar5_ax, 
                        orientation = 'horizontal',
                        extend = 'max',
                        #ticks = [0,4,8,12,16,20,24,28,32,36]
                            ) 
        cbar5.ax.tick_params(labelsize=10)
        cbar5.set_label(unit_zooplankton, fontsize=10)
        
        MesoC_volsum_text = str(round(MesoC_volsum[0],2))
        MesoC_text_all = r'$\Sigma$ '+MesoC_volsum_text+' Pg C'
        m5.text(0.7,0.03,MesoC_text_all, transform=m5.transAxes, color='white', fontsize=10, fontweight='bold')

        
        # Macrozooplankton
        if phaeo_path.is_file():
            axis = axes[3,0]
        else:
            axis = axes[2,1]
        m6 = axis
        if macro_path.is_file():
            f6 = pf.subplot(mesh, fig, m6, [MacroC_layersum[:]], levels = levels_macrozooplankton,
                            units = unit_phytoplankton, mapproj = mapproj, cmap = cmap, cmap_extension = cmap_extension, box = box)
            m6.set_title('Macrozooplankton', fontsize=12)

            if phaeo_path.is_file():
                cbar6_ax = fig.add_axes([0.135, 0.115, 0.35, 0.01])  # left, bottom, width, height
            else:
                cbar6_ax = fig.add_axes([0.535, 0.315, 0.35, 0.01])  # left, bottom, width, height
            cbar6 = fig.colorbar(f6,
                            cax = cbar6_ax, 
                            orientation = 'horizontal',
                            extend = 'max',
                            #ticks = [0,4,8,12,16,20,24,28,32,36]
                                ) 
            cbar6.ax.tick_params(labelsize=10)
            cbar6.set_label(unit_zooplankton, fontsize=10)
            
            MacroC_volsum_text = str(round(MacroC_volsum[0],2))
            MacroC_text_all = r'$\Sigma$ '+MacroC_volsum_text+' Pg C'
            m6.text(0.7,0.03,MacroC_text_all, transform=m6.transAxes, color='white', fontsize=10, fontweight='bold')
  
        else:
            axis.remove()
        
        
        if(self.savefig == True):
            plt.savefig(self.savepath+self.runname+'_Biomass_maps_'+'_'+str(self.fyear)+'to'+str(self.lyear)+'.png', 
                    dpi = 300, bbox_inches='tight')

        plt.show()
        

           
        